# Tensorflow Hub Model with Six Percent Data Augmenetation (Top 10 Diagnosis)

In [1]:
import pickle, random
import numpy as np

In [2]:
import pandas as pd

In [3]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [4]:
winogender_sentences = "all_sentences.tsv"

In [5]:
winogender_occupations = "occupations-stats.tsv"

In [6]:
import csv

wino_occupations = []

In [7]:
with open(winogender_occupations, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        wino_occupations.append(lines["occupation"])

In [8]:
# wino_occupations

In [9]:
len(wino_occupations)

60

In [10]:
len(set(wino_occupations))

60

### Load Text Classification Hub Model

In [11]:
import tensorflow as tf

In [12]:
rand_range = range(1, 11)

In [13]:
# my_model directory
# !ls ../saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls ../saved_model/six_percent_models/

my_model_txt_classifier_hub_with_six_percent_extra_data_rand_1
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_10
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_2
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_3
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_4
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_5
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_6
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_7
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_8
my_model_txt_classifier_hub_with_six_percent_extra_data_rand_9


In [14]:
!ls ../saved_model/six_percent_models/my_model_txt_classifier_hub_with_six_percent_extra_data_rand_1

assets            keras_metadata.pb saved_model.pb    variables


In [15]:
percent_mode = "six"
path_start = "../saved_model/" + str(percent_mode) + "_percent_models/my_model_txt_classifier_hub_with_" + str(percent_mode) + "_percent_extra_data_rand_"
path_start

'../saved_model/six_percent_models/my_model_txt_classifier_hub_with_six_percent_extra_data_rand_'

## Test on Repaired Input Tokens

### A. Direct gender noun comparisons (e.g. My boyfriend/My girlfriend)

In [16]:
mode = "direct-gender-noun"

In [17]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [18]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [19]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [20]:
noun_error = {**noun_error1, **noun_error2}

In [21]:
# noun_error 

In [22]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [23]:
noun_error1

['my aunt',
 'my daughter',
 'my mother',
 'my wife',
 'this girl',
 'my sister',
 'this woman',
 'my girlfriend']

In [24]:
noun_error2

['my uncle',
 'my son',
 'my father',
 'my brother',
 'my husband',
 'this boy',
 'this man',
 'my boyfriend']

In [25]:
noun_error = get_sorted_dict(noun_error)

In [26]:
noun_error = list(noun_error)

In [27]:
input1, input2, input3 = [],[], []

In [28]:
input_pairs, input_truple = [], []

In [29]:
occupation_list = []

In [30]:
i = 0
tmp, tmp2 ="", ""

In [31]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [32]:
f_input, m_input, neut_input = "", "", ""

In [33]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [34]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [35]:
# fairness_errors = []
# all_inputs = []
# count = 0

In [36]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [37]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [38]:
len(all_inputs)

3000

In [39]:
len(fairness_errors)

131

In [40]:
len(fairness_errors_all_rands)

10

In [41]:
fairness_errors_all_rands

[138, 75, 88, 152, 76, 132, 219, 336, 146, 131]

In [42]:
sum(fairness_errors_all_rands)

1493

In [43]:
sum(fairness_errors_all_rands)/10

149.3

In [44]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [51]:
sum(all_rand_inputs)

30000

In [46]:
sum(all_rand_inputs)/10

3000.0

In [47]:
A_errors = sum(fairness_errors_all_rands)

In [48]:
A_inputs = sum(all_rand_inputs)

### B. Random gender noun comparisons (e.g. My boyfriend/My mother)

In [52]:
mode = "random-gender-noun"

In [53]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [54]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [55]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [56]:
noun_error = {**noun_error1, **noun_error2}

In [57]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [58]:
noun_error1

['my aunt',
 'my girlfriend',
 'my daughter',
 'my wife',
 'my mother',
 'this woman',
 'this girl',
 'my sister',
 'my mom']

In [59]:
noun_error2

['my brother',
 'my father',
 'this boy',
 'my boyfriend',
 'my uncle',
 'my husband',
 'this man',
 'my dad',
 'my son']

In [60]:
noun_error = get_sorted_dict(noun_error)

In [61]:
noun_error = list(noun_error)

In [62]:
input1, input2, input3 = [],[], []

In [63]:
input_pairs, input_truple = [], []

In [64]:
occupation_list = []

In [65]:
i = 0
tmp, tmp2 ="", ""

In [66]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [67]:
f_input, m_input, neut_input = "", "", ""

In [68]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = ""
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [69]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [70]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [71]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [72]:
len(all_inputs)

3000

In [73]:
len(fairness_errors)

135

In [74]:
len(fairness_errors_all_rands)

10

In [75]:
fairness_errors_all_rands

[121, 108, 69, 149, 66, 148, 183, 271, 103, 135]

In [76]:
sum(fairness_errors_all_rands)

1353

In [77]:
sum(fairness_errors_all_rands)/10

135.3

In [78]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [79]:
sum(all_rand_inputs)

30000

In [80]:
sum(all_rand_inputs)/10

3000.0

In [81]:
B_errors = sum(fairness_errors_all_rands)

In [82]:
B_inputs = sum(all_rand_inputs)

### C. Test for Indirect Gender Bias, i.e. Occupational Bias

In [83]:
mode = "gender-occupation-noun"

In [84]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [85]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [86]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [87]:
noun_error = {**noun_error1, **noun_error2}

In [88]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [89]:
noun_error1

['the writer',
 'the teacher',
 'the cleaner',
 'the auditor',
 'the counselor',
 'the nurse',
 'the the paralegal',
 'the editor',
 'the tailor',
 'the clerk']

In [90]:
noun_error2

['the cook',
 'the analyst',
 'the engineer',
 'the mechanic',
 'the technician',
 'the supervisor',
 'the salesperson',
 'the farmer',
 'the chief',
 'the lawyer']

In [91]:
noun_error = get_sorted_dict(noun_error)

In [92]:
noun_error = list(noun_error)

In [93]:
input1, input2, input3 = [],[], []

In [94]:
input_pairs, input_truple = [], []

In [95]:
occupation_list = []

In [96]:
i = 0
tmp, tmp2 ="", ""

In [97]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [98]:
f_input, m_input, neut_input = "", "", ""

In [99]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [100]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [101]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [102]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [103]:
len(all_inputs)

3000

In [104]:
len(fairness_errors)

260

In [105]:
len(fairness_errors_all_rands)

10

In [106]:
fairness_errors_all_rands

[154, 130, 115, 169, 181, 120, 168, 257, 301, 260]

In [107]:
sum(fairness_errors_all_rands)

1855

In [108]:
sum(fairness_errors_all_rands)/10

185.5

In [109]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [110]:
sum(all_rand_inputs)

30000

In [111]:
sum(all_rand_inputs)/10

3000.0

In [112]:
c_errors = sum(fairness_errors_all_rands)

In [113]:
C_inputs = sum(all_rand_inputs)

### D. Test for   for Indirect Gender Bias, i.e. Name Bias

In [114]:
mode = "gender-name-noun"

In [115]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [116]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [117]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [118]:
noun_error = {**noun_error1, **noun_error2}

In [119]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [120]:
noun_error1

['Deborah',
 'Sharon',
 'Kimberly',
 'Carol',
 'Michelle',
 'Karen',
 'Sandra',
 'Laura',
 'Nancy',
 'Barbara']

In [121]:
noun_error2

['Christopher',
 'Joshua',
 'Thomas',
 'James',
 'Timothy',
 'Mark',
 'Robert ',
 'George',
 'Edward',
 'Anthony']

In [122]:
noun_error = get_sorted_dict(noun_error)

In [123]:
noun_error = list(noun_error)

In [124]:
input1, input2, input3 = [],[], []

In [125]:
input_pairs, input_truple = [], []

In [126]:
occupation_list = []

In [127]:
i = 0
tmp, tmp2 ="", ""

In [128]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [129]:
f_input, m_input, neut_input = "", "", ""

In [130]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [131]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [132]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [133]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [134]:
len(all_inputs)

3000

In [135]:
len(fairness_errors)

186

In [136]:
len(fairness_errors_all_rands)

10

In [137]:
fairness_errors_all_rands

[221, 171, 147, 318, 229, 156, 358, 258, 265, 186]

In [138]:
sum(fairness_errors_all_rands)

2309

In [139]:
sum(fairness_errors_all_rands)/10

230.9

In [140]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [141]:
sum(all_rand_inputs)

30000

In [142]:
sum(all_rand_inputs)/10

3000.0

In [143]:
D_errors = sum(fairness_errors_all_rands)

In [144]:
D_inputs = sum(all_rand_inputs)

### E. Test for   for Indirect Racial Bias, i.e. Name Bias

In [145]:
mode = "racial-name-noun"

In [146]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [147]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [148]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [149]:
noun_error = {**noun_error1, **noun_error2}

In [150]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [151]:
noun_error1

['Josh',
 'Melanie',
 'Justin',
 'Stephanie',
 'Harry',
 'Jack',
 'Katie',
 'Nancy',
 'Amanda',
 'Ellen']

In [152]:
noun_error2

['Malik',
 'Lamar',
 'Jasmine',
 'Jamel',
 'Nichelle',
 'Latisha',
 'Darnell',
 'Leroy',
 'Alonzo',
 'Shereen']

In [153]:
noun_error = get_sorted_dict(noun_error)

In [154]:
noun_error = list(noun_error)

In [155]:
input1, input2, input3 = [],[], []

In [156]:
input_pairs, input_truple = [], []

In [157]:
occupation_list = []

In [158]:
i = 0
tmp, tmp2 ="", ""

In [159]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [160]:
f_input, m_input, neut_input = "", "", ""

In [161]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [162]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [163]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [164]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [165]:
len(all_inputs)

3000

In [166]:
len(fairness_errors)

158

In [167]:
len(fairness_errors_all_rands)

10

In [168]:
fairness_errors_all_rands

[186, 164, 136, 189, 209, 160, 245, 182, 229, 158]

In [169]:
sum(fairness_errors_all_rands)

1858

In [170]:
sum(fairness_errors_all_rands)/10

185.8

In [171]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [172]:
sum(all_rand_inputs)

30000

In [173]:
sum(all_rand_inputs)/10

3000.0

In [174]:
E_errors = sum(fairness_errors_all_rands)

In [175]:
E_inputs = sum(all_rand_inputs)

### F. Test for Neutral (Sentiment) Sentences

In [176]:
mode = "neutral-sentiments"

In [177]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [178]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [179]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [180]:
noun_error = {**noun_error1, **noun_error2}

In [181]:
# noun_error 

In [182]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [183]:
noun_error1

['my aunt',
 'my daughter',
 'my wife',
 'my mother',
 'my girlfriend',
 'this girl',
 'this woman',
 'my sister',
 'my mom',
 'the writer']

In [184]:
noun_error2

['my son',
 'my uncle',
 'my father',
 'my husband',
 'my brother',
 'this boy',
 'this man',
 'my boyfriend',
 'my dad',
 'the cook']

In [185]:
noun_error = get_sorted_dict(noun_error)

In [186]:
noun_error = list(noun_error)

In [187]:
input1, input2, input3 = [],[], []

In [188]:
input_pairs, input_truple = [], []

In [189]:
occupation_list = []

In [190]:
i = 0
tmp, tmp2 ="", ""

In [191]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [192]:
f_input, m_input, neut_input = "", "", ""

In [193]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [194]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [195]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [196]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [197]:
len(all_inputs)

3000

In [198]:
len(fairness_errors)

199

In [199]:
len(fairness_errors_all_rands)

10

In [200]:
fairness_errors_all_rands

[130, 127, 93, 129, 50, 137, 210, 349, 199, 199]

In [201]:
sum(fairness_errors_all_rands)

1623

In [202]:
sum(fairness_errors_all_rands)/10

162.3

In [203]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [204]:
sum(all_rand_inputs)

30000

In [205]:
sum(all_rand_inputs)/10

3000.0

In [206]:
F_errors = sum(fairness_errors_all_rands)

In [207]:
F_inputs = sum(all_rand_inputs)

In [215]:
F_errors

1623

In [208]:
all_inputs = A_inputs + B_inputs + C_inputs + D_inputs + E_inputs + F_inputs

In [210]:
all_errors = A_errors + B_errors + c_errors + D_errors + E_errors + F_errors

In [211]:
all_inputs

180000

In [212]:
all_errors

10491

In [213]:
all_inputs/6

30000.0

In [214]:
all_errors/6

1748.5

In [216]:
A_errors

1493

In [217]:
all_errors_list = [A_errors, B_errors, c_errors, D_errors, E_errors, F_errors]

In [218]:
all_errors_list

[1493, 1353, 1855, 2309, 1858, 1623]

In [219]:
all_inputs_list = [A_inputs, B_inputs, C_inputs, D_inputs, E_inputs, F_inputs]

In [220]:
all_inputs_list

[30000, 30000, 30000, 30000, 30000, 30000]